In [36]:
import pandas as pd
import os.path

data = pd.read_csv(os.getcwd() + '/steam_games2.csv', warn_bad_lines=True, error_bad_lines=False)
print(len(Data))

C:\Users\Lemond Wyatt\AppData\Local\Temp\ipykernel_9552\944804859.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(os.getcwd() + '/steam_games2.csv', warn_bad_lines=True, error_bad_lines=False)
C:\Users\Lemond Wyatt\AppData\Local\Temp\ipykernel_9552\944804859.py:4: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(os.getcwd() + '/steam_games2.csv', warn_bad_lines=True, error_bad_lines=False)


40833


In [37]:
list(data.columns)

['url',
 'types',
 'name',
 'desc_snippet',
 'recent_reviews',
 'all_reviews',
 'release_date',
 'developer',
 'publisher',
 'popular_tags',
 'game_details',
 'languages',
 'achievements',
 'genre',
 'game_description',
 'mature_content',
 'minimum_requirements',
 'recommended_requirements',
 'original_price',
 'discount_price']

In [38]:
print(data['popular_tags'])


0        FPS,Gore,Action,Demons,Shooter,First-Person,Gr...
1        Survival,Shooter,Multiplayer,Battle Royale,PvP...
2        Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...
3        Survival,Zombies,Open World,Multiplayer,PvP,Ma...
4        Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...
                               ...                        
40828                                    Casual,Simulation
40829                                    Casual,Simulation
40830    RPG,Indie,Strategy,Software,Turn-Based,Fantasy...
40831                                               Action
40832                                        RPG,Adventure
Name: popular_tags, Length: 40833, dtype: object


In [53]:
filtered_data = data[data['name'].str.contains('warframe', na=False)]
print(filtered_data)

Empty DataFrame
Columns: [url, types, name, desc_snippet, recent_reviews, all_reviews, release_date, developer, publisher, popular_tags, game_details, languages, achievements, genre, game_description, mature_content, minimum_requirements, recommended_requirements, original_price, discount_price]
Index: []
